In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from src.template import template_assessment

/home/andrey/anaconda3/envs/aml_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Model and tokenizer names{
model_name = "google/gemma-1.1-2b-it" #"mistralai/Mistral-7B-Instruct-v0.2" #"google/gemma-1.1-2b-it" # #"google/gemma-1.1-2b-it" #"google/gemma-7b-it"

# Load the model and tokenizer
device = torch.device("cuda:1")
model = AutoModelForCausalLM.from_pretrained(model_name).eval().to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.75it/s]


In [7]:
model_params = {
    "max_new_tokens": 700,
    #"temperature": 0.1,
}
COURSE_NAME = "Big Data Technologies and Analytics"#"Advanced Machine Learning"
COURSE_DESCRIPTION = "This course covers the following concepts: Advanced distributed data organization; Advanced distributed data processing." #""This course covers the following concepts: Advanced topics in machine learning; To develop research interest in the theory and application of machine learning."
PREREQUISITES = {
    "PREREQUISITE_SUBJECTS": [],
    "PREREQUISITE_TOPICS": []
}
# COURSE_TOPICS = [
#         {
#             "Section": "Deep Learning Architectures and Optimization",
#             "Topics within the section": [
#                 "Convolutional Neural Networks",
#                 "Autoencoders",
#                 "Generative Adversarial Networks",
#                 "Training Deep Learning Models",
#                 "Optimization Techniques for Deep Learning"
#             ]
#         },
#         {
#             "Section": "Unsupervised Learning Techniques and Applications",
#             "Topics within the section": [
#                 "K-means Clustering",
#                 "Hierarchical Clustering",
#                 "Dimensionality Reduction Techniques",
#                 "Anomaly Detection",
#                 "Generative Adversarial Networks"
#             ]
#         },
#         {
#             "Section": "Large-Scale Machine Learning and Distributed Learning",
#             "Topics within the section": [
#                 "Distributed Training",
#                 "Parallel Computing Frameworks",
#                 "Scaling Machine Learning Models",
#                 "Federated Learning",
#                 "Local Outlier Detection"
#             ]
#         },
#         {
#             "Section": "Emerging Trends in Machine Learning",
#             "Topics within the section": [
#                 "Reinforcement Learning",
#                 "Natural Language Processing",
#                 "Generative Adversarial Networks",
#                 "Diffusion Models",
#                 "Bayesian Deep Learning"
#             ]
#         }
# ]
COURSE_TOPICS = [
    {
      "Section": "Introduction",
      "Topics within the section": [
        "What is Big Data",
        "Characteristics of Big Data",
        "Technologies",
        "Virtualization and cloud computing"
      ]
    },
    {
      "Section": "File systems and resource managers",
      "Topics within the section": [
        "HDFS",
        "YARN"
      ]
    },
    {
      "Section": "Batch Processing",
      "Topics within the section": [
        "Distributed batch processing",
        "MapReduce model",
        "Applications",
        "Tasks management",
        "Patterns"
      ]
    },
    {
      "Section": "Stream Processing",
      "Topics within the section": [
        "CAP theorem",
        "Distributed storage and computation",
        "Distributed Stream Processing",
        "Usage patterns"
      ]
    },
    {
      "Section": "Analytics",
      "Topics within the section": [
        "Architecture",
        "Use cases",
        "SparkML",
        "GraphX"
      ]
    }
]
content = f"{template_assessment}\nCOURSE: {COURSE_NAME}\nDESCRIPTION: {COURSE_DESCRIPTION}\nCOURSE_TOPICS: {COURSE_TOPICS}\nFINAL_ASSESSMENT:"
message = [ 
    {
        "role": "user",
        "content": content,
    },
]
prompt = tokenizer.apply_chat_template(
        message, tokenize=False, add_generation_prompt=True
    )
input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
generated_ids = model.generate(
    input_ids=input_ids.to(device),
    **model_params,
)
response = tokenizer.decode(generated_ids[0])

In [8]:
print(response[len(prompt):])

{
  "Sections": [
    {
      "Section": "Introduction",
      "Topics within the section": [
        "What is big data and its characteristics?",
        "What are the technologies and tools used in big data analytics?",
        "How does virtualization and cloud computing impact big data processing?"
      ]
    },
    {
      "Section": "File systems and resource managers",
      "Topics within the section": [
        "HDFS: Distributed file system for big data storage",
        "YARN: Resource manager for big data processing"
      ]
    },
    {
      "Section": "Batch Processing",
      "Topics within the section": [
        "Distributed batch processing: Coordinating tasks across multiple nodes",
        "MapReduce model: Programming paradigm for parallel processing of large datasets",
        "Applications of batch processing in big data"
      ]
    },
    {
      "Section": "Stream Processing",
      "Topics within the section": [
        "CAP theorem: Scalable streaming data

In [9]:
course_topics = response[len(prompt):].split('"FINAL_ASSESSMENT": ')[1]
start_index = course_topics.index("{")
end_index = course_topics.rindex("}") + 1
print(course_topics[start_index: end_index])

IndexError: list index out of range

In [11]:
with open('bd_gemma_assessment.json', 'w', encoding='utf-8') as f:
    f.write(f'{response[len(prompt):]}')

In [7]:
import ast
res = {"Final assessment": ast.literal_eval(course_topics[start_index: end_index])}
import json
with open('aml_gemma_assessment.json', 'w', encoding='utf-8') as f:
    json.dump(res, f, ensure_ascii=False, indent=4)

SyntaxError: closing parenthesis ']' does not match opening parenthesis '{' on line 1 (<unknown>, line 28)